In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
import math
%matplotlib inline

In [2]:
%%time
mydata = pd.read_csv('NY property data_zip_stories.csv')

Wall time: 4.47 s


1. Refill the FULLVAL, AVLAND, AVTOT, add new classifier: the maximum of (LTFRONT, LTDEPTH, BLDFRONT and BLDDEPTH) 

2. Refill the LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH, add new classifier: the bin_value of the mean of (FULLVAL, AVLAND, AVTOT)


一、FULLVAL

Turn 0s into NAN

In [3]:
mydata['FULLVAL']=mydata['FULLVAL'].replace(0, np.nan)
mydata['FULLVAL'].isna().value_counts()

False    1057987
True       13007
Name: FULLVAL, dtype: int64

Turn STORIES to INT

In [4]:
data = mydata
data['TAXCLASS']=mydata['TAXCLASS'].astype('str')
data['ZIP']=mydata['ZIP'].astype('str')
data['STORIES']=mydata['STORIES'].astype(np.float64)

data['STORIES']=round(data['STORIES'])
data['STORIES']=mydata['STORIES'].astype('str')

Consider the maximum of (LTFRONT, LTDEPTH, BLDFRONT and BLDDEPTH) as classifier

In [5]:
data['new'] = data[['LTFRONT','LTDEPTH','BLDFRONT','BLDDEPTH']].max(axis=1)
bins = [i*50 for i in range(201)]
groups = [i for i in range(200)]
data['area_level'] = pd.cut(data['new'], bins, labels=groups)
data['area_level']=data['area_level'].replace(np.nan,0)
data['area_level']=mydata['area_level'].astype('str')

use TAXCLASS STORIES area_level ZIP

In [6]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['FULLVAL'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanFULLVAL'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanFULLVAL':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanFULLVAL_y']].groupby('group')['meanFULLVAL_y'].mean()
del data10['meanFULLVAL_x']

mydata['FULLVAL'].fillna(data10['meanFULLVAL_y'],inplace=True ) 

mydata['FULLVAL'].isna().value_counts()


False    1063973
True        7021
Name: FULLVAL, dtype: int64

Under 10 use TAXCLASS STORIES area_level

In [7]:

data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['FULLVAL'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanFULLVAL'] = data['group'].apply(get_matched_value)

dt1= data.groupby(['group']).agg({'meanFULLVAL':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanFULLVAL_y']].groupby('group')['meanFULLVAL_y'].mean()
del data10['meanFULLVAL_x']

mydata['FULLVAL'].fillna(data10['meanFULLVAL_y'],inplace=True ) 

mydata['FULLVAL'].isna().value_counts()


False    1068303
True        2691
Name: FULLVAL, dtype: int64

Under 10 use STORIES,area_level

In [8]:
data['group'] = data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['FULLVAL'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanFULLVAL'] = data['group'].apply(get_matched_value)

dt1= data.groupby(['group']).agg({'meanFULLVAL':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanFULLVAL_y']].groupby('group')['meanFULLVAL_y'].mean()
del data10['meanFULLVAL_x']

mydata['FULLVAL'].fillna(data10['meanFULLVAL_y'],inplace=True ) 

mydata['FULLVAL'].isna().value_counts()

False    1070902
True          92
Name: FULLVAL, dtype: int64

STILL HAVE NULL —— use stories

In [9]:
data['group'] = data['STORIES']
table = data.groupby(['group'])['FULLVAL'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanFULLVAL'] = data['group'].apply(get_matched_value)

dt1= data.groupby(['group']).agg({'meanFULLVAL':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanFULLVAL_y']].groupby('group')['meanFULLVAL_y'].mean()
del data10['meanFULLVAL_x']

mydata['FULLVAL'].fillna(data10['meanFULLVAL_y'],inplace=True ) 

mydata['FULLVAL'].isna().value_counts()

False    1070992
True           2
Name: FULLVAL, dtype: int64

STILL HAVE NULL —— use TAXCLASS

In [10]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['FULLVAL'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanFULLVAL'] = data['group'].apply(get_matched_value)

dt1= data.groupby(['group']).agg({'meanFULLVAL':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanFULLVAL_y']].groupby('group')['meanFULLVAL_y'].mean()
del data10['meanFULLVAL_x']

mydata['FULLVAL'].fillna(data10['meanFULLVAL_y'],inplace=True ) 

mydata['FULLVAL'].isna().value_counts()

False    1070994
Name: FULLVAL, dtype: int64

二、AVLAND

In [11]:
mydata['AVLAND']=mydata['AVLAND'].replace(0, np.nan)


In [12]:
mydata['AVLAND'].isna().value_counts()

False    1057985
True       13009
Name: AVLAND, dtype: int64

In [13]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVLAND'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVLAND'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVLAND':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVLAND_y']].groupby('group')['meanAVLAND_y'].mean()
del data10['meanAVLAND_x']

mydata['AVLAND'].fillna(data10['meanAVLAND_y'],inplace=True ) 

mydata['AVLAND'].isna().value_counts()


False    1063973
True        7021
Name: AVLAND, dtype: int64

In [14]:
data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVLAND'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVLAND'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVLAND':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVLAND_y']].groupby('group')['meanAVLAND_y'].mean()
del data10['meanAVLAND_x']

mydata['AVLAND'].fillna(data10['meanAVLAND_y'],inplace=True ) 

mydata['AVLAND'].isna().value_counts()

False    1068303
True        2691
Name: AVLAND, dtype: int64

In [15]:
data['group'] = data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVLAND'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVLAND'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVLAND':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVLAND_y']].groupby('group')['meanAVLAND_y'].mean()
del data10['meanAVLAND_x']

mydata['AVLAND'].fillna(data10['meanAVLAND_y'],inplace=True ) 

mydata['AVLAND'].isna().value_counts()

False    1070702
True         292
Name: AVLAND, dtype: int64

In [16]:
data['group'] = data['STORIES']
table = data.groupby(['group'])['AVLAND'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVLAND'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVLAND':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVLAND_y']].groupby('group')['meanAVLAND_y'].mean()
del data10['meanAVLAND_x']

mydata['AVLAND'].fillna(data10['meanAVLAND_y'],inplace=True ) 

mydata['AVLAND'].isna().value_counts()

False    1070992
True           2
Name: AVLAND, dtype: int64

In [17]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['AVLAND'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVLAND'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVLAND':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVLAND_y']].groupby('group')['meanAVLAND_y'].mean()
del data10['meanAVLAND_x']

mydata['AVLAND'].fillna(data10['meanAVLAND_y'],inplace=True ) 

mydata['AVLAND'].isna().value_counts()

False    1070994
Name: AVLAND, dtype: int64

三、AVTOT

In [18]:
mydata['AVTOT']=mydata['AVTOT'].replace(0, np.nan)

In [19]:
data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVTOT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVTOT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVTOT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVTOT_y']].groupby('group')['meanAVTOT_y'].mean()
del data10['meanAVTOT_x']

mydata['AVTOT'].fillna(data10['meanAVTOT_y'],inplace=True ) 

mydata['AVTOT'].isna().value_counts()


False    1063973
True        7021
Name: AVTOT, dtype: int64

In [20]:
data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVTOT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVTOT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVTOT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVTOT_y']].groupby('group')['meanAVTOT_y'].mean()
del data10['meanAVTOT_x']

mydata['AVTOT'].fillna(data10['meanAVTOT_y'],inplace=True ) 

mydata['AVTOT'].isna().value_counts()


False    1068303
True        2691
Name: AVTOT, dtype: int64

In [21]:
data['group'] = data['STORIES']+'.'+data['area_level']
table = data.groupby(['group'])['AVTOT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVTOT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVTOT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVTOT_y']].groupby('group')['meanAVTOT_y'].mean()
del data10['meanAVTOT_x']

mydata['AVTOT'].fillna(data10['meanAVTOT_y'],inplace=True ) 

mydata['AVTOT'].isna().value_counts()


False    1070702
True         292
Name: AVTOT, dtype: int64

In [22]:
data['group'] = data['STORIES']
table = data.groupby(['group'])['AVTOT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVTOT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVTOT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVTOT_y']].groupby('group')['meanAVTOT_y'].mean()
del data10['meanAVTOT_x']

mydata['AVTOT'].fillna(data10['meanAVTOT_y'],inplace=True ) 

mydata['AVTOT'].isna().value_counts()

False    1070992
True           2
Name: AVTOT, dtype: int64

In [23]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['AVTOT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanAVTOT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanAVTOT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanAVTOT_y']].groupby('group')['meanAVTOT_y'].mean()
del data10['meanAVTOT_x']

mydata['AVTOT'].fillna(data10['meanAVTOT_y'],inplace=True ) 

mydata['AVTOT'].isna().value_counts()

False    1070994
Name: AVTOT, dtype: int64

Fill LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH

LTFRONT

USE value level=mean(FULLVAL, AVLAND, AVTOT) as classifier

In [24]:
data['meanvalue'] = data[['FULLVAL','AVLAND','AVTOT']].mean(axis=1)


In [25]:
data['meanvalvalue'] = data[['FULLVAL','AVLAND','AVTOT']].mean(axis=1)
bins = [i*1000 for i in range(200)]
bins.extend([a*10000+200000 for a in range(80)])
bins.extend([i*100000+1000000 for i in range(90)])
bins.extend([i*1000000+10000000 for i in range(90)])
bins.extend([i*1000000+100000000 for i in range(90)])
bins.extend([i*10000000+1000000000 for i in range(90)])
bins.extend([i*100000000+10000000000 for i in range(90)])


In [26]:
groups = [i for i in range(len(bins)-1)]
data['val_level'] = pd.cut(data['meanvalvalue'], bins, labels=groups)
data['val_level']=data['val_level'].replace(np.nan,0)
data['val_level']=mydata['val_level'].astype('str')

In [31]:
mydata['LTFRONT']=mydata['LTFRONT'].replace(0, np.nan)
mydata['LTFRONT'].isna().value_counts()

False    901886
True     169108
Name: LTFRONT, dtype: int64

In [32]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    916876
True     154118
Name: LTFRONT, dtype: int64

In [33]:
data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    1010629
True       60365
Name: LTFRONT, dtype: int64

In [34]:
data['group'] = data['TAXCLASS']+data['val_level']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    1069274
True        1720
Name: LTFRONT, dtype: int64

In [35]:
data['group'] = data['STORIES']+data['val_level']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    1070954
True          40
Name: LTFRONT, dtype: int64

In [37]:
data['group'] = data['val_level']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    1070993
True           1
Name: LTFRONT, dtype: int64

In [39]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['LTFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTFRONT_y']].groupby('group')['meanLTFRONT_y'].mean()
del data10['meanLTFRONT_x']

mydata['LTFRONT'].fillna(data10['meanLTFRONT_y'],inplace=True ) 

mydata['LTFRONT'].isna().value_counts()

False    1070994
Name: LTFRONT, dtype: int64

LTDEPTH

In [40]:
mydata['LTDEPTH']=mydata['LTDEPTH'].replace(0, np.nan)
mydata['LTDEPTH'].isna().value_counts()

False    900866
True     170128
Name: LTDEPTH, dtype: int64

In [41]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    916080
True     154914
Name: LTDEPTH, dtype: int64

In [42]:
data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    1009508
True       61486
Name: LTDEPTH, dtype: int64

In [43]:
data['group'] = data['TAXCLASS']+data['val_level']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    1069271
True        1723
Name: LTDEPTH, dtype: int64

In [44]:
data['group'] = data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    1070951
True          43
Name: LTDEPTH, dtype: int64

In [45]:
data['group'] = data['val_level']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    1070993
True           1
Name: LTDEPTH, dtype: int64

In [46]:
data['group'] = data['STORIES']
table = data.groupby(['group'])['LTDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanLTDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanLTDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanLTDEPTH_y']].groupby('group')['meanLTDEPTH_y'].mean()
del data10['meanLTDEPTH_x']

mydata['LTDEPTH'].fillna(data10['meanLTDEPTH_y'],inplace=True ) 

mydata['LTDEPTH'].isna().value_counts()

False    1070994
Name: LTDEPTH, dtype: int64

BLDFRONT

In [47]:
mydata['BLDFRONT']=mydata['BLDFRONT'].replace(0, np.nan)
mydata['BLDFRONT'].isna().value_counts()

False    842179
True     228815
Name: BLDFRONT, dtype: int64

In [49]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    851998
True     218996
Name: BLDFRONT, dtype: int64

In [50]:

data['group'] = data['TAXCLASS']+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    949764
True     121230
Name: BLDFRONT, dtype: int64

In [51]:
data['group'] = data['TAXCLASS']+'.'data['val_level']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    1040434
True       30560
Name: BLDFRONT, dtype: int64

In [52]:
data['group'] = data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    1070313
True         681
Name: BLDFRONT, dtype: int64

In [53]:
data['group'] =data['val_level']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    1070983
True          11
Name: BLDFRONT, dtype: int64

In [55]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['BLDFRONT'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDFRONT'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDFRONT':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=0').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDFRONT_y']].groupby('group')['meanBLDFRONT_y'].mean()
del data10['meanBLDFRONT_x']

mydata['BLDFRONT'].fillna(data10['meanBLDFRONT_y'],inplace=True ) 

mydata['BLDFRONT'].isna().value_counts()

False    1070994
Name: BLDFRONT, dtype: int64

BLDDEPTH

In [56]:
mydata['BLDDEPTH']=mydata['BLDDEPTH'].replace(0, np.nan)
mydata['BLDDEPTH'].isna().value_counts()

False    842141
True     228853
Name: BLDDEPTH, dtype: int64

In [57]:

data['group'] = data['TAXCLASS']+'.'+data['ZIP']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    851945
True     219049
Name: BLDDEPTH, dtype: int64

In [58]:
data['group'] = data['TAXCLASS']+'.'+data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    949638
True     121356
Name: BLDDEPTH, dtype: int64

In [59]:
data['group'] = data['TAXCLASS']+'.'+data['val_level']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    1040503
True       30491
Name: BLDDEPTH, dtype: int64

In [60]:
data['group'] = data['STORIES']+'.'+data['val_level']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    1070314
True         680
Name: BLDDEPTH, dtype: int64

In [61]:
data['group'] = data['val_level']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    1070931
True          63
Name: BLDDEPTH, dtype: int64

In [62]:
data['group'] = data['TAXCLASS']
table = data.groupby(['group'])['BLDDEPTH'].mean()

table = round(table)
def get_matched_value(string):
    val = table.loc[string]
    return val


data['meanBLDDEPTH'] = data['group'].apply(get_matched_value)


dt1= data.groupby(['group']).agg({'meanBLDDEPTH':'mean','RECORD':'count'})
dt10=dt1.query('RECORD>=10').reset_index()

data10=data.merge(dt10, on =['group'], how = 'left')
data10[['group','meanBLDDEPTH_y']].groupby('group')['meanBLDDEPTH_y'].mean()
del data10['meanBLDDEPTH_x']

mydata['BLDDEPTH'].fillna(data10['meanBLDDEPTH_y'],inplace=True ) 

mydata['BLDDEPTH'].isna().value_counts()

False    1070994
Name: BLDDEPTH, dtype: int64

Check unusual points in our previous result

In [27]:
mydata[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][mydata['RECORD']==684704]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
684703,41423.0,814.0,814.0,11385.0,2.0


In [28]:
mydata[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][mydata['RECORD']==776306]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
776305,109055.0,19109.0,49075.0,11385.0,1.0


In [29]:
mydata[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][mydata['RECORD']==821853]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
821852,227597.0,12656.0,48912.0,11432.0,4.0


In [63]:
mydata[['LTFRONT','LTDEPTH','BLDDEPTH','BLDFRONT','FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][mydata['RECORD']==917942]

,LTFRONT,LTDEPTH,BLDDEPTH,BLDFRONT,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
917941,4910.0,3028.0,81.0,380.0,374019883.0,1.792809e+09,4.668309e+09,11422.0,3.0


In [66]:
mydata.columns

Index(['Unnamed: 0', 'RECORD', 'BBLE', 'B', 'BLOCK', 'LOT', 'EASEMENT',
       'OWNER', 'BLDGCL', 'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES',
       'FULLVAL', 'AVLAND', 'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR',
       'ZIP', 'EXMPTCL', 'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2',
       'EXLAND2', 'EXTOT2', 'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'new',
       'area_level', 'group', 'meanFULLVAL', 'meanAVLAND', 'meanAVTOT',
       'meanvalue', 'meanvalvalue', 'val_level', 'meanLTFRONT', 'meanLTDEPTH',
       'meanBLDFRONT', 'meanBLDDEPTH'],
      dtype='object')

In [80]:

del mydata['new']
del mydata['area_level']
del mydata['group']
del mydata['meanFULLVAL']
del mydata['meanAVLAND']
del mydata['meanAVTOT']
del mydata['meanvalvalue']
del mydata['val_level']
del mydata['meanLTFRONT']
del mydata['meanLTDEPTH']
del mydata['meanBLDFRONT']
del mydata['meanBLDDEPTH']


In [81]:
pd.DataFrame(mydata).to_csv('NY property data_filled_Revised.csv')